In [1]:
import os
os.path.abspath("")

'/home/yuxiangliao/PhD/workspace/VSCode_workspace/structured_reporting/notebook'

## Load data

In [2]:
import pandas as pd
REPORT_PATH = "/home/yuxiangliao/PhD/data/mimic_cxr_reports_core.json"
df = pd.read_json(REPORT_PATH,orient="records",lines=True)
print(df)

id_list = df.loc[:,'sid'].to_list()
findings_list = df.loc[:,'findings'].to_list()
impression_list = df.loc[:,'impression'].to_list()
pfi_list = df.loc[:,'provisional_findings_impression'].to_list()
fai_list = df.loc[:,'findings_and_impression'].to_list()

DATA_SIZE = len(id_list)

              pid        sid  \
0       p10000032  s50414267   
1       p10000032  s53189527   
2       p10000032  s53911762   
3       p10000032  s56699142   
4       p10000764  s57375967   
...           ...        ...   
227830  p19999442  s58708861   
227831  p19999733  s57132437   
227832  p19999987  s55368167   
227833  p19999987  s58621812   
227834  p19999987  s58971208   

                                                 findings  \
0       There is no focal consolidation, pleural effus...   
1       The cardiac, mediastinal and hilar contours ar...   
2       Single frontal view of the chest provided. \n ...   
3       The lungs are clear of focal consolidation, pl...   
4       PA and lateral views of the chest provided.   ...   
...                                                   ...   
227830  ET tube ends 4.7 cm above the carina.  NG tube...   
227831  The lungs are clear, and the cardiomediastinal...   
227832  There has been interval extubation and improve...   
22783

# Check CoreNLP

In [61]:
!ls $CORENLP_HOME

build.xml				  LIBRARY-LICENSES
corenlp.sh				  LICENSE.txt
CoreNLP-to-HTML.xsl			  Makefile
ejml-core-0.39.jar			  patterns
ejml-core-0.39-sources.jar		  pom-java-11.xml
ejml-ddense-0.39.jar			  pom-java-17.xml
ejml-ddense-0.39-sources.jar		  pom.xml
ejml-simple-0.39.jar			  protobuf-java-3.19.2.jar
ejml-simple-0.39-sources.jar		  README.txt
input.txt				  RESOURCE-LICENSES
input.txt.out				  sample-project-pom.xml
input.txt.xml				  SemgrexDemo.java
istack-commons-runtime-3.0.7.jar	  ShiftReduceDemo.java
istack-commons-runtime-3.0.7-sources.jar  slf4j-api.jar
javax.activation-api-1.2.0.jar		  slf4j-simple.jar
javax.activation-api-1.2.0-sources.jar	  stanford-corenlp-4.5.0.jar
javax.json-api-1.0-sources.jar		  stanford-corenlp-4.5.0-javadoc.jar
javax.json.jar				  stanford-corenlp-4.5.0-models.jar
jaxb-api-2.4.0-b180830.0359.jar		  stanford-corenlp-4.5.0-sources.jar
jaxb-api-2.4.0-b180830.0359-sources.jar   StanfordCoreNlpDemo.java
jaxb-impl-2.4.0-b180830.0438.jar	  StanfordDependenci

# CoreNLP

# CoreNLPClient

In [3]:
from hydra import compose, initialize
from omegaconf import OmegaConf
import hydra

config_path = "../src/config/"
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path=config_path, version_base=None)
config = compose(config_name="nlp_ensemble")

print(OmegaConf.to_yaml(config))

machine:
  work_dir: /home/yuxiangliao/PhD/workspace/VSCode_workspace/structured_reporting
  fast_coref_dir: /home/yuxiangliao/PhD/workspace/git_clone_repos/fast-coref/
work_dir: ${machine.work_dir}
src_dir: ${work_dir}/src
output_dir: ${work_dir}/output
log_dir: ${work_dir}/logs/${hydra.job.config_name}
logging_level: INFO
ignore_source_path: ${work_dir}/config/ignore/common_ignore
fast_coref_dir: ${machine.fast_coref_dir}
coref_scorer_dir: ${machine.fast_coref_dir}/coref_resources/reference-coreference-scorers
name_style:
  mimic_cxr:
    section_name:
      PID: pid
      SID: sid
      FINDINGS: findings
      IMPRESSION: impression
      PFI: provisional_findings_impression
      FAI: findings_and_impression
      CLINICAL_INFO: clinical_information
      PROCEDURE_INFO: procedure_information
      COMPARISON: comparison
      WET_READ: wet_read
      ADDENDUM: addendum
      UNKNOWN: unknown
  corenlp:
    prefix: '[co]'
    statistical_coref_prefix: '[ml]'
    nerual_coref_prefi

In [4]:
import stanza
from stanza.server import CoreNLPClient
import time
import json
COREF_PROPS = {
    'annotators':'tokenize, ssplit, pos, lemma, ner, depparse, coref',
    "coref.algorithm": "statistical",
    'outputFormat': 'json',
}
DCOREF_PROPS = {
    'annotators':'tokenize, ssplit, pos, lemma, ner, parse, dcoref',
    'outputFormat': 'json',
}
NEURAL_PROPS = {
    'annotators':'tokenize, ssplit, pos, lemma, ner, parse, coref',
    'coref.algorithm': 'neural ',
    'outputFormat': 'json',
}
client = CoreNLPClient(memory='4G', threads=8, endpoint='http://localhost:8802', be_quiet=False,output_format="JSON", properties=COREF_PROPS)
client.start()
client.ensure_alive()

2022-09-07 17:30:34 INFO: Writing properties to tmp file: corenlp_server-77a7b1e014114b06.props
2022-09-07 17:30:34 INFO: Starting server with command: java -Xmx4G -cp /home/yuxiangliao/PhD/Stanford_CoreNLP/stanford-corenlp-4.5.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 8802 -timeout 60000 -threads 8 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-77a7b1e014114b06.props -preload -outputFormat JSON
[main] INFO CoreNLP - --- StanfordCoreNLPServer#main() called ---
[main] INFO CoreNLP -     Build: x86_64-conda-linux-gnu
[main] INFO CoreNLP - Server default properties:
			(Note: unspecified annotator properties are English defaults)
			annotators = tokenize, ssplit, pos, lemma, ner, depparse, coref
			coref.algorithm = statistical
			inputFormat = text
			outputFormat = JSON
			prettyPrint = false
			threads = 8
[main] INFO CoreNLP - Threads: 8
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.p

In [5]:
import requests
import pandas as pd

sid = 's52683488'
t = fai_list[id_list.index(sid)]

file_path = os.path.join("/home/yuxiangliao/PhD/workspace/VSCode_workspace/structured_reporting/output/nlp_ensemble", "findings_and_impression", f"{sid}.csv")
file_path = os.path.join("/home/yuxiangliao/Desktop", f"temp_{sid}.csv")
df_from_disk = pd.read_csv(os.path.join("../output/nlp_ensemble/findings_and_impression", f"{sid}.csv"), index_col=0)

request_url = 'http://0.0.0.0:8802/'
corenlp_out_jsonStr = requests.post(request_url, data=t.encode()).text



[pool-1-thread-3] INFO CoreNLP - [/127.0.0.1:47288] API call w/annotators tokenize,pos,lemma,ner,depparse,coref
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator depparse
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator coref


AP radiograph of the chest demonstrates severe pulmonary edema, grossly unchanged since ___ obtained at 00:54 a.m. As compared to the prior study from ___, the extent of the pulmonary edema appears to be minimally progressed but overall unchanged.


[pool-1-thread-3] INFO edu.stanford.nlp.coref.statistical.SimpleLinearClassifier - Loading coref model edu/stanford/nlp/models/coref/statistical/ranking_model.ser.gz ... done [1.1 sec].
[pool-1-thread-3] INFO edu.stanford.nlp.pipeline.CorefMentionAnnotator - Using mention detector type: dependency


In [6]:
import sys
sys.path.append("../src/")
from nlp_ensemble.nlp_processor.corenlp_process import CorenlpUrlProcess, formatCorenlpDocument
from common_utils.nlp_utils import resolveTokenIndices_byPosition
from common_utils.ensemble_utils import load_data_bySection
from common_utils.nlp_utils import align_byIndex_individually_nestedgruop, align_byIndex_individually_withData_dictInList, align_coref_groups_in_conll_format, align_byIndex_individually_withData_noOverlap
from common_utils.ensemble_utils import load_data_bySection
from common_utils.nlp_utils import align, getTokenOffset
from nlp_ensemble.nlp_processor.spacy_process import SpacyProcess, init_spacy

In [8]:
import spacy
SPACY_NLP = spacy.load("en_core_web_md",disable=['ner'])

text_tuples = [(t, {"sid": sid})]
spacy_output_list = []
for doc, context in SPACY_NLP.pipe(text_tuples, as_tuples=True):
    sid = context["sid"]
    spacy_output_list.append({"sid": sid, "doc": doc})

spacy_nametyle = config.name_style.spacy.column_name
for spacy_output in spacy_output_list:
    sid = spacy_output["sid"]
    doc = spacy_output["doc"]
    df_spacy = pd.DataFrame(
        {
            spacy_nametyle["token"]: [str(tok.text) for tok in doc],
            spacy_nametyle["token_offset"]: [int(i) for i in getTokenOffset(doc.text, doc)],
            spacy_nametyle["sentence_group"]: [int(i) for i in align(len(doc), doc.sents)],
            spacy_nametyle["noun_chunk"]: [int(i) for i in align(len(doc), doc.noun_chunks)],
            spacy_nametyle["lemma"]: [str(tok.lemma_) for tok in doc],
            spacy_nametyle["pos_core"]: [f"[{tok.pos_}]{spacy.explain(tok.pos_)}" for tok in doc],
            spacy_nametyle["pos_feature"]: [f"[{tok.tag_}]{spacy.explain(tok.tag_)}" for tok in doc],
            spacy_nametyle["dependency_tag"]: [f"[{tok.dep_}]{spacy.explain(tok.dep_)}" for tok in doc],
            spacy_nametyle["dependency_head"]: [f"{tok.head.text}|{tok.head.i}" for tok in doc],
            spacy_nametyle["dependency_children"]: [[f"{child.text}|{child.i}" for child in tok.children] for tok in doc],
            spacy_nametyle["morphology"]: [str(tok.morph) for tok in doc],
            spacy_nametyle["is_alpha"]: [bool(tok.is_alpha) for tok in doc],
            spacy_nametyle["is_stop"]: [bool(tok.is_stop) for tok in doc],
            spacy_nametyle["is_pronoun"]: [bool(tok.pos_ == "PRON") for tok in doc],
            spacy_nametyle["trailing_space"]: [bool(tok.whitespace_) for tok in doc],
        }
    )

In [9]:
df_spacy

,[sp]token,[sp]token_offset,[sp]sentence_group,[sp]noun_chunk,[sp]lemma,[sp]pos_core,[sp]pos_feature,[sp]dependency,[sp]dep_head|idx,[sp]dep_children|idx,[sp]morphology,[sp]is_alpha,[sp]is_stop,[sp]is_pronoun,[sp]trailing_space
0,AP,0,0,0,AP,[PROPN]proper noun,"[NNP]noun, proper singular",[compound]compound,radiograph|1,[],Number=Sing,True,False,False,True
1,radiograph,3,0,0,radiograph,[NOUN]noun,"[NN]noun, singular or mass",[nsubj]nominal subject,demonstrates|5,"[AP|0, of|2]",Number=Sing,True,False,False,True
2,of,14,0,-1,of,[ADP]adposition,"[IN]conjunction, subordinating or preposition",[prep]prepositional modifier,radiograph|1,[chest|4],,True,True,False,True
3,the,17,0,1,the,[DET]determiner,[DT]determiner,[det]determiner,chest|4,[],Definite=Def|PronType=Art,True,True,False,True
4,chest,21,0,1,chest,[NOUN]noun,"[NN]noun, singular or mass",[pobj]object of preposition,of|2,[the|3],Number=Sing,True,False,False,True
5,demonstrates,27,0,-1,demonstrate,[VERB]verb,"[VBZ]verb, 3rd person singular present",[ROOT]root,demonstrates|5,"[radiograph|1, edema|8, a.m.|19]",Number=Sing|Person=3|Tense=Pres|VerbForm=Fin,True,False,False,True
6,severe,40,0,2,severe,[ADJ]adjective,"[JJ]adjective (English), other noun-modifier (...",[amod]adjectival modifier,edema|8,[],Degree=Pos,True,False,False,True
7,pulmonary,47,0,2,pulmonary,[ADJ]adjective,"[JJ]adjective (English), other noun-modifier (...",[amod]adjectival modifier,edema|8,[],Degree=Pos,True,False,False,True
8,edema,57,0,2,edema,[NOUN]noun,"[NN]noun, singular or mass",[dobj]direct object,demonstrates|5,"[severe|6, pulmonary|7, ,|9, unchanged|11, obt...",Number=Sing,True,False,False,False
9,",",62,0,-1,",",[PUNCT]punctuation,"[,]punctuation mark, comma",[punct]punctuation,edema|8,[],PunctType=Comm,False,False,False,True


In [10]:
section_name_cfg = config.name_style.mimic_cxr.section_name
batch_process_cfg = config.batch_process
output_section_cfg = config.output.section
nlp_cfg = config.nlp_properties.corenlp
request_url = config.nlp_properties.corenlp.request_url
spacy_nametyle = config.name_style.spacy.column_name
corenlp_nametyle = config.name_style.corenlp.column_name

coref_server_name = "dcoref"

corenlp_json = json.loads(corenlp_out_jsonStr)
df_base = df_spacy
tokenOffset_base = df_base.loc[:, spacy_nametyle["token_offset"]].tolist()
referTo_spacy, tokenTotalNum, sentenceGroups, corefMetionGroups_withData, corefGroups, dependency_list, depPlus_list, depPlusPlus_list = formatCorenlpDocument(tokenOffset_base, corenlp_json)
df_corenlp_rowsNum = tokenTotalNum
df_corenlp = None
df_corenlp = pd.DataFrame(
    {
        corenlp_nametyle["align_to_spacy"]: [int(i) for i in referTo_spacy],
        corenlp_nametyle["token"]: [str(token["originalText"]) for sentence in corenlp_json["sentences"] for token in sentence["tokens"]],
        corenlp_nametyle["token_offset"]: [str(token["characterOffsetBegin"]) for sentence in corenlp_json["sentences"] for token in sentence["tokens"]],
        corenlp_nametyle["sentence_group"]: [int(i) for i in sentenceGroups],
        corenlp_nametyle["lemma"]: [str(token["lemma"]) for sentence in corenlp_json["sentences"] for token in sentence["tokens"]],
        corenlp_nametyle["pos"]: [str(token["pos"]) for sentence in corenlp_json["sentences"] for token in sentence["tokens"]],
        corenlp_nametyle["dependency"]: [str(i) for i in align_byIndex_individually_withData_dictInList(df_corenlp_rowsNum, dependency_list)],
        corenlp_nametyle["dependency+"]: [str(i) for i in align_byIndex_individually_withData_dictInList(df_corenlp_rowsNum, depPlus_list)],
        corenlp_nametyle["dependency++"]: [str(i) for i in align_byIndex_individually_withData_dictInList(df_corenlp_rowsNum, depPlusPlus_list)],
        corenlp_nametyle[coref_server_name + "_mention"]: [str(i) for i in align_byIndex_individually_withData_noOverlap(df_corenlp_rowsNum, corefMetionGroups_withData)],
        corenlp_nametyle[coref_server_name + "_group"]: [str(i) for i in align_byIndex_individually_nestedgruop(df_corenlp_rowsNum, corefGroups)],
        corenlp_nametyle[coref_server_name + "_group_conll"]: [str(i) for i in align_coref_groups_in_conll_format(df_corenlp_rowsNum, corefGroups)],
    },
)

In [ ]:
df_base

,[sp]token,[sp]token_offset,[sp]sentence_group,[sp]noun_chunk,[sp]lemma,[sp]pos_core,[sp]pos_feature,[sp]dependency,[sp]dep_head|idx,[sp]dep_children|idx,...,[sp]trailing_space,[co]token,[co]pos,[co]lemma,[co]dep_tag|dep_head|head_index,[co](+)dep_tag|dep_head|head_index,[co](++)dep_tag|dep_head|head_index,[co][ml]corefId|corefType,[co][ml]coref_group,[co][ml]coref_group_conll
0,AP,0,0,0,AP,[PROPN]proper noun,"[NNP]noun, proper singular",[compound]compound,radiograph|1,[],...,True,AP,NN,ap,['compound|radiograph|1'],['compound|radiograph|1'],['compound|radiograph|1'],-1,-1,-1
1,radiograph,3,0,0,radiograph,[NOUN]noun,"[NN]noun, singular or mass",[nsubj]nominal subject,demonstrates|5,"['AP|0', 'of|2']",...,True,radiograph,NN,radiograph,['nsubj|demonstrates|5'],['nsubj|demonstrates|5'],['nsubj|demonstrates|5'],-1,-1,-1
2,of,14,0,-1,of,[ADP]adposition,"[IN]conjunction, subordinating or preposition",[prep]prepositional modifier,radiograph|1,['chest|4'],...,True,of,IN,of,['case|chest|4'],['case|chest|4'],['case|chest|4'],-1,-1,-1
3,the,17,0,1,the,[DET]determiner,[DT]determiner,[det]determiner,chest|4,[],...,True,the,DT,the,['det|chest|4'],['det|chest|4'],['det|chest|4'],-1,-1,-1
4,chest,21,0,1,chest,[NOUN]noun,"[NN]noun, singular or mass",[pobj]object of preposition,of|2,['the|3'],...,True,chest,NN,chest,['nmod|radiograph|1'],['nmod:of|radiograph|1'],['nmod:of|radiograph|1'],-1,-1,-1
5,demonstrates,27,0,-1,demonstrate,[VERB]verb,"[VBZ]verb, 3rd person singular present",[ROOT]root,demonstrates|5,"['radiograph|1', 'edema|8', 'a.m.|19']",...,True,demonstrates,VBZ,demonstrate,['ROOT|demonstrates|5'],['ROOT|demonstrates|5'],['ROOT|demonstrates|5'],-1,-1,-1
6,severe,40,0,2,severe,[ADJ]adjective,"[JJ]adjective (English), other noun-modifier (...",[amod]adjectival modifier,edema|8,[],...,True,severe,JJ,severe,['amod|edema|8'],['amod|edema|8'],['amod|edema|8'],2|NOMINAL,[1],['(1']
7,pulmonary,47,0,2,pulmonary,[ADJ]adjective,"[JJ]adjective (English), other noun-modifier (...",[amod]adjectival modifier,edema|8,[],...,True,pulmonary,JJ,pulmonary,['amod|edema|8'],['amod|edema|8'],['amod|edema|8'],2|NOMINAL,[1],-1
8,edema,57,0,2,edema,[NOUN]noun,"[NN]noun, singular or mass",[dobj]direct object,demonstrates|5,"['severe|6', 'pulmonary|7', ',|9', 'unchanged|...",...,False,edema,NN,edema,['obj|demonstrates|5'],['obj|demonstrates|5'],['obj|demonstrates|5'],2|NOMINAL,[1],-1
9,",",62,0,-1,",",[PUNCT]punctuation,"[,]punctuation mark, comma",[punct]punctuation,edema|8,[],...,True,",",",",",",['punct|edema|8'],['punct|edema|8'],['punct|edema|8'],2|NOMINAL,[1],-1


In [11]:
len(df_corenlp.index)
df_corenlp

,[co]align_to_spacy,[co]token,[co]token_offset,[co]sentence_group,[co]lemma,[co]pos,[co]dep_type|dep_head|head_index,[co](+)dep_type|dep_head|head_index,[co](++)dep_type|dep_head|head_index,[co][rb]corefId|corefType,[co][rb]coref_group,[co][rb]coref_group_conll
0,0,AP,0,0,ap,NN,['compound|radiograph|1'],['compound|radiograph|1'],['compound|radiograph|1'],-1,-1,-1
1,1,radiograph,3,0,radiograph,NN,['nsubj|demonstrates|5'],['nsubj|demonstrates|5'],['nsubj|demonstrates|5'],-1,-1,-1
2,2,of,14,0,of,IN,['case|chest|4'],['case|chest|4'],['case|chest|4'],-1,-1,-1
3,3,the,17,0,the,DT,['det|chest|4'],['det|chest|4'],['det|chest|4'],-1,-1,-1
4,4,chest,21,0,chest,NN,['nmod|radiograph|1'],['nmod:of|radiograph|1'],['nmod:of|radiograph|1'],-1,-1,-1
5,5,demonstrates,27,0,demonstrate,VBZ,['ROOT|demonstrates|5'],['ROOT|demonstrates|5'],['ROOT|demonstrates|5'],-1,-1,-1
6,6,severe,40,0,severe,JJ,['amod|edema|8'],['amod|edema|8'],['amod|edema|8'],2|NOMINAL,[1],['(1']
7,7,pulmonary,47,0,pulmonary,JJ,['amod|edema|8'],['amod|edema|8'],['amod|edema|8'],2|NOMINAL,[1],-1
8,8,edema,57,0,edema,NN,['obj|demonstrates|5'],['obj|demonstrates|5'],['obj|demonstrates|5'],2|NOMINAL,[1],-1
9,9,",",62,0,",",",",['punct|edema|8'],['punct|edema|8'],['punct|edema|8'],2|NOMINAL,[1],-1


In [ ]:
df1 = pd.DataFrame({'k1': ['a', 'b'],
                    'v1': [1, 2]},index=[0,0])
df2 = pd.DataFrame({'k2': ['c', 'd'],
                    'v2': [3, 4]},index=[0,0])

In [ ]:
pd.concat([df1, df2], axis=1)

,k1,v1,k2,v2
0,a,1,c,3
0,b,2,d,4


In [ ]:
df_concat = pd.concat([df_corenlp,df_corenlp_coref],axis=1)
df_concat

,[co]token,[co]pos,[co]lemma,[co]dep_tag|dep_head|head_index,[co](+)dep_tag|dep_head|head_index,[co](++)dep_tag|dep_head|head_index,[co][rb]corefId|corefType,[co][rb]coref_group,[co][rb]coref_group_conll
0,AP,NN,ap,['compound|radiograph|1'],['compound|radiograph|1'],['compound|radiograph|1'],0|NOMINAL,[0],['(0']
1,radiograph,NN,radiograph,['nsubj|demonstrates|5'],['nsubj|demonstrates|5'],['nsubj|demonstrates|5'],0|NOMINAL,[0],-1
2,of,IN,of,['case|chest|4'],['case|chest|4'],['case|chest|4'],0|NOMINAL,[0],-1
3,the,DT,the,['det|chest|4'],['det|chest|4'],['det|chest|4'],1|NOMINAL,"[0, 1]",['(1']
4,chest,NN,chest,['nmod|radiograph|1'],['nmod:of|radiograph|1'],['nmod:of|radiograph|1'],1|NOMINAL,"[0, 1]","['0)', '1)']"
5,demonstrates,VBZ,demonstrate,['ROOT|demonstrates|5'],['ROOT|demonstrates|5'],['ROOT|demonstrates|5'],-1,-1,-1
6,severe,JJ,severe,['amod|edema|8'],['amod|edema|8'],['amod|edema|8'],-1,-1,-1
7,pulmonary,JJ,pulmonary,['amod|edema|8'],['amod|edema|8'],['amod|edema|8'],-1,-1,-1
8,edema,NN,edema,['obj|demonstrates|5'],['obj|demonstrates|5'],['obj|demonstrates|5'],-1,-1,-1
9,",",",",",",['punct|edema|8'],['punct|edema|8'],['punct|edema|8'],-1,-1,-1


In [ ]:
print(len(df_base.index))
print(len(df_corenlp.index))
print(len(df_concat.index))

47
43
43
43


In [12]:
client.stop()

[Thread-0] INFO CoreNLP - CoreNLP Server is shutting down.


# Stanza

Biomedical models

In [ ]:
# import stanza
# stanza.download('en', package='mimic')
# stanza.download('en', package='radiology')

Stanza processor: https://stanfordnlp.github.io/stanza/pipeline.html#processors

In [ ]:
# from stanza.pipeline.core import DownloadMethod
# import stanza

# processor_dict = {
#     'tokenize': 'mimic', 
#     'pos': 'mimic', 
#     'lemma': 'mimic',
#     'depparse': 'mimic',
#     # 'sentiment':'sstplus', # Sentiment scores of 0, 1, or 2 (negative, neutral, positive).
#     'constituency': 'wsj', # wsj, wsj_bert, wsj_roberta
#     'ner': 'radiology',
# }
# nlp = stanza.Pipeline('en', processors=processor_dict, package=None, 
#                       download_method=DownloadMethod.REUSE_RESOURCES,
#                       verbose=False) # logging_level='WARN'
# doc = nlp(findings_list[8690])
# # print out dependency tree
# print(doc)

Processing Multiple Documents

28s -> 100docs

In [10]:
import stanza
from stanza.pipeline.core import DownloadMethod

processor_dict = {
    'tokenize': 'mimic', 
    'pos': 'mimic', 
    'lemma': 'mimic',
    'depparse': 'mimic',
    # 'sentiment':'sstplus', # Sentiment scores of 0, 1, or 2 (negative, neutral, positive).
    'constituency': 'wsj', # wsj, wsj_bert, wsj_roberta
    'ner': 'radiology',
}

def set_sid(self, value):
    self._sid = value
def get_sid(self):
    return self._sid

nlp = stanza.Pipeline('en', processors=processor_dict, package=None, 
                      download_method=DownloadMethod.REUSE_RESOURCES,
                      verbose=False) # logging_level='WARN'
documents = findings_list[8690:8691] # Documents that we are going to process
# stanza.Document.add_property("sid",getter=get_sid, setter=set_sid)


NER tag
- B，Begin
- I，Intermediate
- E，End
- S，Single
- O，Other

In [11]:
in_docs = []
for id,d in enumerate(documents):
    stanzaDoc = stanza.Document([], text=d)
    stanzaDoc._sid = id_list[id]
    in_docs.append(stanzaDoc)
# in_docs = [stanza.Document([], text=d) for d in documents] # Wrap each document with a stanza.Document object
out_docs = nlp(in_docs) # Call the neural pipeline on this list of documents
# print(out_docs[0]) # The output is also a list of stanza.Document objects, each output corresponding to an input Document object
for doc in out_docs:
    print(f"sid:{doc._sid}")
    print(doc)

sid:s50414267
[
  [
    {
      "id": 1,
      "text": "Compared",
      "lemma": "compare",
      "upos": "VERB",
      "xpos": "VBN",
      "head": 5,
      "deprel": "case",
      "start_char": 0,
      "end_char": 8,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 2,
      "text": "with",
      "lemma": "with",
      "upos": "ADP",
      "xpos": "IN",
      "head": 5,
      "deprel": "case",
      "start_char": 9,
      "end_char": 13,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 3,
      "text": "the",
      "lemma": "the",
      "upos": "DET",
      "xpos": "DT",
      "head": 5,
      "deprel": "det",
      "start_char": 14,
      "end_char": 17,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "prior",
      "lemma": "prior",
      "upos": "ADJ",
      "xpos": "JJ",
      "head": 5,
      "deprel": "amod",
      "start_char": 18,
      "end_char": 23,
 